<a href="https://colab.research.google.com/github/7ZXU/AI/blob/main/HandsOnMachineLearning/HandsOnCh10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**10.2 케라스로 다층 퍼셉트론 구현하기**

##**10.2.1 텐서플로 2 설치**

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
tf.__version__

In [ ]:
keras.__version__

##**10.2.2 시퀀셜 API를 사용하여 이미지 분류기 만들기**

In [ ]:

fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
X_train_full.shape #이미지가 28*28 배열

In [ ]:
X_train_full.dtype # 픽셀강도 정수 0~255

In [ ]:
#검증세트 만들기 #훈련세트 가져오기
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
##255로 나누는 이유 ? 
#경사 하강법으로 훈련하기에 입력 틍성의 스케일 (0~1) 조정
#########################왜 조정해야돼?
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test/255.0

In [ ]:

#클래스 이름 리스트 만들기 #like MINIST label 5
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]

In [ ]:
class_names[y_train[0]]

**시퀀셜 API를 사용하여 모델 만들기**

In [ ]:
model = keras.models.Sequential() #
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [ ]:
model.summary()

**모델 컴파일**

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="sgd", 
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

**모델학습곡선**

In [ ]:
#에포크마다 측정한 평균 훈련손실, 정확도
#에포크 종료 시점마다 측정한 평균 검증 손실과 정확도 
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1) #수직축 범위 0~1 설정
plt.show()

**모델 사용해 예측 만들기 : 확률 출력**

In [ ]:
X_new = X_test[3:6] #테스트 세트에서 새로운 샘플 받아오기
y_proba = model.predict(X_new) #새로운 샘플 예측
y_proba.round(2) 
#X_test[3] 클래스 1일 확률 : 100%
#X_test[4] 클래스 6일 확률 : 70%

**모델 사용해 예측 만들기 : 가장 높은 확률만 출력**

In [ ]:
import numpy as np 

y_pred = model.predict_classes(X_new)
y_pred
np.array(class_names)[y_pred]

##**10.2.3 시퀀셜 API 사용하여 회귀용 다층 퍼셉트론 만들기**

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

#split train, test
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)

#train, validation split
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [ ]:
model = keras.models.Sequential([
                                 keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
                                 keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

#evaluate model by test case
mse_test = model.evaluate(X_test, y_test)

#t새로운 샘플
X_new = X_test[:3] 
y_pred = model.predict(X_new)


##**10.2.4 함수형 API를 사용해 복잡한 모델 만들기**

In [ ]:
input_=keras.layers.Input(shape=X_train.shape[1:])
hidden1=keras.layers.Dense(30, activation="relu")(input_)
hidden2=keras.layers.Dense(30, activation="relu")(hidden1)
concat=keras.layers.Concatenate()([input_, hidden2])
output=keras.layers.Dense(1)(concat) #활성화함수 없는 출력층
model=keras.Model(inputs=[input_], outputs=[output])

**여러 개의 입력 다루기**

In [ ]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [ ]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3)) #lr

X_train_A, X_train_B = X_train[:, :5], X_train[: ,2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20, 
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

**여러 출력 다루기**

In [ ]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

In [ ]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd") 
#손실을 모두 더해서 최종 손실 구함
#주 출력에 더 큰 가중치 

In [ ]:
history = model.fit(
    [X_train_A, X_train_B], [y_train, y_train], epochs=20,
    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid])
)
#주 출력 = 보조 출력
#따라서 동일한 레이블 사용



In [ ]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test]
)

#개별손실 
#총손실

In [ ]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])
print(y_pred_main, y_pred_aux)

##**10.2.5 서브클래싱 API로 동적 모델 만들기**

**클래스로 함수형 API 기능 수행**

In [ ]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) #표준 매개변수 처리
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel()

##**10.2.6 모델 저장과 복원**

In [ ]:
model.save_weights("my_keras_model.h5")

##**10.2.7 콜백 사용하기**

**ModelCheckpoint**

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5")
#매 에포크 끝에서 호출됨
#훈련하는 동안 일정한 간격으로 모델의 체크 포인트 저장
history = model.fit(X_train, y_train, epochs=10, callbacks=[checkpoint_cb])

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
#최상의 검증 세트 점수에서만 모델 저장 
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), 
                    callbacks=[checkpoint_cb])
model = keras.models.load_model("my_keras_model.h5")
#최상의 모델로 복원

**EarlyStopping Call back**

조기 종료 구현

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True
)
#patience = 반복횟수
#검증세트에 대한 점수가 향상되지 않으면 훈련 정지
#최상의 가중치 저장

history = model.fit(X_train, y_train, epochs=100, 
                    validation_data = (X_valid, y_valid), 
                    callbacks = [checkpoint_cb, early_stopping_cb])
#체크포인트저장콜백 : 컴퓨터 문제 일으키는 경우 대비
#조기종료콜백 : 시간 및 자원 낭비 방지

**사용자 정의 콜백**

In [ ]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))
        #검증손실과 훈련손실 비율 측정 
        #과대적합 감지

##**10.2.8 텐서 보드를 사용해 시각화하기**

In [ ]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")
#os.curdir : 현재 디렉토리
#텐서보드 로그를 위해 사용

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
#현재 날짜와 시간 사용
#실행할 때마다 다른 서브디렉터리 경로 생성
#테스트하는 하이퍼파라미터값과 같은 추가적 정보도 로그디렉터리 이름으로 사용

run_logdir = get_run_logdir()
#ex) ./my_logs/run_2019_06_07-15_15_22



In [ ]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid), 
                    callbacks=[tensorboard_cb])

#**10.3 신경망 하이퍼파라미터 튜닝하기**

##**10.3.1 은닉층 개수**

##**10.3.2 은닉층의 뉴런 개수**

##**10.3.3 학습률, 배치 크기 그리고 다른 하이퍼파라미터**